In [1]:
import pyspark
from pyspark.sql import SparkSession, types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/03/04 22:28:30 WARN Utils: Your hostname, DESKTOP-IQOU5OM resolves to a loopback address: 127.0.1.1; using 172.20.246.72 instead (on interface eth0)
24/03/04 22:28:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 22:28:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.5.0'

In [7]:
import pandas as pd

In [8]:
df_fhv = pd.read_csv('data/raw/fhv/2019/10/fhv_tripdata_2019-10.csv.gz')

In [11]:
df_fhv_copy = df_fhv.head(10)

In [13]:
spark.createDataFrame(df_fhv_copy).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [28]:
fhv_schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
types.StructField('pickup_datetime', types.TimestampType(), True), 
types.StructField('dropOff_datetime', types.TimestampType(), True), 
types.StructField('PUlocationID', types.IntegerType(), True), 
types.StructField('DOlocationID', types.IntegerType(), True), 
types.StructField('SR_Flag', types.DoubleType(), True), 
types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [29]:
df_fhv_result = spark.read \
                .option("header","true") \
                .schema(fhv_schema) \
                .csv('./data/raw/fhv/2019/10/*')

In [30]:
df_fhv_result.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [31]:
output_path = f'data/pq/fhv/2019/10'

In [33]:
df_fhv_result.repartition(6).write.parquet(output_path,mode='overwrite')

Record count on the 15th of October

In [34]:
df_fhv_result.createOrReplaceTempView('fhv')

In [35]:
df_fhv_15th = spark.sql(
    """
    SELECT COUNT(*) FROM fhv

    WHERE pickup_datetime > '2019-10-15 00:00:00'
    AND pickup_datetime <= '2019-10-16 00:00:00'
    """
)

In [36]:
df_fhv_15th.show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



Longest Trip distance (Hours)

In [37]:
from pyspark.sql import functions as F

In [40]:
df_fhv_result = df_fhv_result.withColumn('trip_duration',(F.col('dropoff_datetime').cast("long") - F.col('pickup_datetime').cast("long"))/3600)

In [45]:
max_duration = df_fhv_result.agg(F.max("trip_duration")).first()[0]

In [46]:
max_duration

631152.5

In [52]:
df_fhv_result.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|       trip_duration|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|                 0.2|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013| 0.03138888888888889|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   NULL|                B00014| 0.42694444444444446|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   NULL|                B00014|0.021666666666666667|
|            

Least Frequent Pickup Zone

In [51]:
# Merge zones table with the fhv table

df_zones = spark.read.parquet('zones/')
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [61]:
df_result = df_fhv_result.join(df_zones.withColumnRenamed("Zone","pickup_zone"),df_fhv_result.PUlocationID == df_zones.LocationID,how='left')
df_result.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+----------+-------+---------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|       trip_duration|LocationID|Borough|    pickup_zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+----------+-------+---------------+------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|                 0.2|       264|Unknown|             NV|         N/A|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013| 0.03138888888888889|       264|Unknown|             NV|         N/A|
|              B00014|2019-10-01 00:11:43|2019-10-01 00

In [70]:
least_frequented_zone = df_result.groupBy("pickup_zone").count().orderBy("count").first()[0]
least_frequented_zone

'Jamaica Bay'

In [55]:
df_result.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+----------+--------+--------------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|       trip_duration|LocationID| Borough|        dropoff_zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------------------+----------+--------+--------------------+------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   NULL|                B00009|                 0.2|       264| Unknown|                  NV|         N/A|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   NULL|                B00013| 0.03138888888888889|       264| Unknown|                  NV|         N/A|
|              B00014|201